In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

# Dataset
X, y = load_breast_cancer(return_X_y=True)

# Espaço de busca
space = [
    # RandomForest
    Integer(50, 200, name="n_estimators_rf"),
    Integer(2, 15, name="max_depth_rf"),

    # SVM
    Real(0.01, 10.0, name="C_svm"),
    Categorical(["linear", "rbf"], name="kernel_svm"),

    # XGBoost
    Integer(50, 200, name="n_estimators_xgb"),
    Real(0.01, 0.3, name="learning_rate_xgb"),
    Integer(2, 10, name="max_depth_xgb"),

    # Voting type
    Categorical(["soft", "hard"], name="voting_type")
]

# Função objetivo
@use_named_args(space)
def objective(**params):
    rf = RandomForestClassifier(
        n_estimators=params["n_estimators_rf"],
        max_depth=params["max_depth_rf"],
        random_state=42
    )

    svc = SVC(
        C=params["C_svm"],
        kernel=params["kernel_svm"],
        probability=True,  # necessário para voting="soft"
        random_state=42
    )

    xgb = XGBClassifier(
        n_estimators=params["n_estimators_xgb"],
        learning_rate=params["learning_rate_xgb"],
        max_depth=params["max_depth_xgb"],
        eval_metric="logloss",
        random_state=42
    )

    # Meta-modelo: Logistic Regression
    meta_lr = LogisticRegression(max_iter=1000, random_state=42)

    # Stacking (RF + SVM como base, LogisticRegression como meta-modelo)
    stacking = StackingClassifier(
        estimators=[("rf", rf), ("svc", svc),("xgb", xgb)],
        final_estimator=meta_lr,
        passthrough=True
    )

    # Voting (RF + SVM + XGB)
    voting = VotingClassifier(
        estimators=[("rf", rf), ("svc", svc), ("xgb", xgb)],
        voting=params["voting_type"]
    )

    # Avaliar ambos
    scores_stack = cross_val_score(stacking, X, y, cv=5, scoring="accuracy")
    scores_vote = cross_val_score(voting, X, y, cv=5, scoring="accuracy")

    # Combinação (50% stacking, 50% voting)
    return -(0.5 * np.mean(scores_stack) + 0.5 * np.mean(scores_vote))

# Rodar otimização
res = gp_minimize(objective, space, n_calls=40, random_state=42)

print("Melhor score combinado:", -res.fun)
print("Melhores hiperparâmetros:", res.x)
print("Nomeados:", dict(zip([dim.name for dim in space], res.x)))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [20:48:14] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [20:48:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [20:48:18] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [20:48:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: l